#Donwload a ImageCollection 

## 1._ Install libraries

In [2]:
!pip3 install earthengine-api;
!pip3 install geemap;
!pip3 install ee;

In [5]:
import sys
sys.path

['/Users/hn/Documents/00_GitHub/Ag/NASA/Python_codes/GEE_Notebooks/00_scratches',
 '/Users/hn/opt/anaconda3/lib/python38.zip',
 '/Users/hn/opt/anaconda3/lib/python3.8',
 '/Users/hn/opt/anaconda3/lib/python3.8/lib-dynload',
 '',
 '/Users/hn/opt/anaconda3/lib/python3.8/site-packages',
 '/Users/hn/opt/anaconda3/lib/python3.8/site-packages/aeosa',
 '/Users/hn/opt/anaconda3/lib/python3.8/site-packages/locket-0.2.1-py3.8.egg',
 '/Users/hn/opt/anaconda3/lib/python3.8/site-packages/IPython/extensions',
 '/Users/hn/.ipython']

In [3]:
!pip3 install ipykernel --upgrade

In [4]:
import ee

ModuleNotFoundError: No module named 'ee'

## 2._ Establish connection

In [2]:
!earthengine authenticate

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code&code_challenge=tm7sM3bnVbF9E7TGV47CUCzczKhHhwFi60eEFea3iC4&code_challenge_method=S256

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: ^C
Traceback (most recent call last):
  File "/opt/anaconda3/bin/earthengine", line 10, in <module>
    sys.exit(main())
  File "/opt/anaconda3/lib/python3.9/site-packages/ee/cli/eecli.py", line 84, in main
    _run_command()
  File "/opt/anaconda3/lib/python3.9/site-packa

In [3]:

service_account = 'h.noorazar@gmail.com'
credentials = ee.ServiceAccountCredentials(service_account, 'privatekey.json')
ee.Initialize(credentials)

ModuleNotFoundError: No module named 'ee'

In [ ]:
import ee
try:
  ee.Initialize()
  print('The Earth Engine package initialized successfully!')
except ee.EEException as e:
  print('The Earth Engine package failed to initialize!')
except:
  print("Unexpected error:", sys.exc_info()[0])
  raise

## 3._ Example

In [ ]:
import ee

In [ ]:
pt = ee.Geometry.Point([-2.40986111110000012, 26.76033333330000019])
region = pt.buffer(10)

col = ee.ImageCollection('LANDSAT/LC08/C01/T1')\
        .filterDate('2015-01-01','2015-04-30')\
        .filterBounds(region)
        

## 4._ Define bands 

In [ ]:
bands = ['B4','B5'] #Change it!


##5.  Function to pass to Iterate

In [ ]:
def accumulate(image,img):
  name_image = image.get('system:index')
  image = image.select([0],[name_image])
  cumm = ee.Image(img).addBands(image)
  return cumm


In [ ]:
for band in bands:
  col_band = col.map(lambda img: img.select(band)\
                               .set('system:time_start', img.get('system:time_start'))\
                               .set('system:index', img.get('system:index')))
  #  ImageCollection to List           
  col_list = col_band.toList(col_band.size())

  #  Define the initial value for iterate.
  base = ee.Image(col_list.get(0))
  base_name = base.get('system:index')
  base = base.select([0], [base_name])
  
  #  Eliminate the image 'base'.
  new_col = ee.ImageCollection(col_list.splice(0,1))

  img_cummulative = ee.Image(new_col.iterate(accumulate,base))

  task = ee.batch.Export.image.toDrive(
      image = img_cummulative.clip(region),
      folder = 'landsat',
      fileNamePrefix = band,
      scale = 30).start()  
  
  print('Export Image '+ band+ ' was submitted, please wait ...')

In [ ]:
img_cummulative.bandNames().getInfo()